# Group 22 Project Proposal

## Gender Wage Disparities in the United States: A Comparative Analysis

### Table of Contents

- [Introduction](#Introduction)
- [Preliminary Results](#Preliminary-Results)
    - [Data wrangling](#Data-wrangling)
    - [Plotting of relevant variables](#Plotting-of-relevant-variables)
- [Methods](#Methods)
- [References](#References)

# Introduction

The proposed project aims to investigate gender wage disparities in the United States using a dataset containing information on various demographic and employment-related variables. The primary question we seek to answer is whether there is a significant difference in wages between male and female workers. We will focus on the hourly wage as our main variable of interest to compare across the two genders. By analyzing this dataset, we aim to shed light on existing gender wage gaps and assess the potential impact of such disparities.

# Preliminary Results

## Data wrangling

In [2]:
library(infer)
library(repr)
library(readr)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ dplyr   1.0.10
✔ tibble  3.1.8      ✔ stringr 1.4.1 
✔ tidyr   1.2.1      ✔ forcats 0.5.2 
✔ purrr   0.3.4      
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


We have successfully imported and cleaned the dataset, ensuring that the data is in a tidy format for analysis. We have also plotted the raw data, specifically examining the distribution of hourly wages for male and female workers. In the preliminary analysis, we observed that there is a clear difference in the distribution of hourly wages between males and females.

To estimate the gender wage disparity, we have computed summary statistics, including the median and standard deviation of hourly wages for both genders. These initial findings show that there is a substantial difference in average hourly wages between male and female workers.

In [17]:
pay_data <- read_csv('https://github.com/AVietCong/stat201-group22-project/raw/main/Gender_Pay_Gap_dataset.csv')
head(pay_data)

Rows: 53790 Columns: 129
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (1): adj_occ2name
dbl (127): year, serial, numprec, gq, region, statefip, metro, metarea, coun...
lgl   (1): farm

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year serial numprec gq region statefip metro metarea county farm ⋯ foodcare
1 2013  4484  1       1  11     50       1       NA       NA  NA   ⋯ 0       
2 2013 21761  3       1  21     39       3     2002       NA  NA   ⋯ 0       
3 2013  7174  4       1  11     44       2     6483       NA  NA   ⋯ 0       
4 2013 58428  1       1  31     12       3     4901    12009  NA   ⋯ 0       
5 2013  2894  3       1  11     33       1       NA       NA  NA   ⋯ 0       
6 2013 56021  1       1  31     12       4     2900    12001  NA   ⋯ 0       
  building sales officeadmin farmer constructextractinstall production
1 0        0     0           0      0                       0         
2 0        0     0           0      1                       0         
3 0        0     0           0      0                       0         
4 0        0     0           0      0                       0         
5 0        0     0           0      0                       0         
6 0        0     0           0      1                       0         
  transport sumadj_occ LEHS
1 0         1          1   
2 0         1          1   
3 0         1          1   
4 0         1          1   
5 1         1          1   
6 0         1          1

The dataset we will be using comprises information on a sample of individuals from the United States, including variables such as sex, race, hourly wage (hrwage), total income (incwage), years of schooling (sch), weeks worked in a year (wkswork1), usual hours worked per week (uhrswork), and other relevant factors. For our analysis, we have narrowed down the dataset to focus only on the variables of interest, namely sex, race, hourly wage, total income, schooling years, weeks worked, and usual hours worked.

In [19]:
pay_data <- select(pay_data, sex, race, hrwage, incwage, sch, wkswork1, uhrswork, potexp) %>%
    mutate(sex = as.factor(sex),
           race = as.factor(race),
           hrwage = as.double(hrwage),
           incwage = as.double(incwage),
           sch = as.double(sch),
           wkswork = as.integer(wkswork1),
           uhrswork = as.integer(uhrswork),
           potexp = as.integer(potexp))
head(pay_data)

sex race hrwage   incwage sch wkswork1 uhrswork wkswork
1 1   1    24.91259 57000   14  52       44       52     
2 1   1    31.00000 62000   13  50       40       50     
3 1   3    21.63461 45000   12  52       40       52     
4 1   1    12.82051 28000   12  52       42       52     
5 1   1    21.25000 42500   12  50       40       50     
6 1   1    21.87500 35000   11  40       40       40

In [20]:
pay_data <- pay_data %>%
    mutate(sex = fct_recode(sex, 'male' = '1', 'female' = '2'),
           race = fct_recode(race, 'White' = '1', 'Black' = '2', 'Hispanic' = '3', 'Other' = '4'),
           education_level = case_when(sch == 0 ~ 'None',
                                       sch < 12 ~ 'Incomplete Schooling', 
                                       sch == 12 ~ 'Highschool',
                                       sch == 13 ~ 'College',
                                       sch == 14 ~ 'Associate Degree',
                                       sch == 15 ~ 'BA',
                                       sch >= 16 ~ 'Postsecondary'))
head(pay_data)

sex  race     hrwage   incwage sch wkswork1 uhrswork wkswork
1 male White    24.91259 57000   14  52       44       52     
2 male White    31.00000 62000   13  50       40       50     
3 male Hispanic 21.63461 45000   12  52       40       52     
4 male White    12.82051 28000   12  52       42       52     
5 male White    21.25000 42500   12  50       40       50     
6 male White    21.87500 35000   11  40       40       40

## Plotting of relevant variables

### Average Potential Experience of Man and Woman

In [10]:
avg_potexp_summary <- pay_data %>%
    group_by(sex) %>%
    summarise(avg_potexp = mean(potexp))
avg_potexp_summary

sex    avg_potexp
1 male   22.64522  
2 female 22.77030

In [11]:
avg_potexp_plot <- ggplot(avg_potexp_summary, aes(x = sex, y = avg_potexp, fill = sex)) + 
    geom_bar(stat = 'identity') + 
    labs(x = 'Gender', y = 'Years of Experience', title = 'Average Years of Experience for Male vs Female') +
    theme(text = element_text(size = 15))

avg_potexp_plot

# Methods

In [44]:
sex_plot <- pay_data |>
    group_by(sex) |>
    summarise(count = n()) |>
    ggplot(aes(x = sex, y = count, fill = sex)) +
    geom_bar(stat = "identity", width = 0.25) +
    theme(text = element_text(size = 20)) +
    labs(x = "Sex", y = "Counts", fill = "Sex")
    
sex_plot

# References